In [2]:
# n_gram 유사도 분석
# 문장 내 단어를 n 개로 나누어 분석
# 연속되는 문장에서 일부 단어만 차례로 확인하다 보니
# 전체 문장을 고려한 언어 모델보다 정확도가 떨어짐
# 보통 n은 1 ~ 5 의 값을 많이 사용함

from konlpy.tag import Komoran

def word_ngram(bow, num_gram) :
    text = tuple(bow)
    ngrams = [text[x:x + num_gram] for x in range(0, len(text))]
    return tuple(ngrams)

def similarity(doc1, doc2) :
    cnt = 0
    for token in doc1 :
        if token in doc2 :
            cnt += 1
    return cnt / len(doc1)

sentence1 = "6월에 뉴턴은 선생님의 제안으로 트리니티에 입학했다."
sentence2 = "6월에 뉴턴은 선생님의 제안으로 대학교에 입학했다."
sentence3 = "나는 맛있는 밥을 뉴턴 선생님과 함께 먹었다."

komoran = Komoran()
bow1 = komoran.nouns(sentence1)
bow2 = komoran.nouns(sentence2)
bow3 = komoran.nouns(sentence3)

doc1 = word_ngram(bow1, 2)
doc2 = word_ngram(bow2, 2)
doc3 = word_ngram(bow3, 2)

print(doc1)
print(doc2)
print(doc3)

r1 = similarity(doc1, doc2)
r2 = similarity(doc1, doc3)
r3 = similarity(doc2, doc3)

print(r1)
print(r2)
print(r3)

(('6월', '뉴턴'), ('뉴턴', '선생님'), ('선생님', '제안'), ('제안', '트리니티'), ('트리니티', '입학'), ('입학',))
(('6월', '뉴턴'), ('뉴턴', '선생님'), ('선생님', '제안'), ('제안', '대학교'), ('대학교', '입학'), ('입학',))
(('밥', '뉴턴'), ('뉴턴', '선생'), ('선생', '님과 함께'), ('님과 함께',))
0.6666666666666666
0.0
0.0


In [3]:
# [ 코사인 유사도 ]
# 단어나 문장을 벡터로 표현한다면 벡터 간 거리나 각도를 이용해 유사성을 파악할 수 있다
# 두 벡터 간 거리를 코사인 각도를 이용해 유사도를 측정한다
# 벡터의 크기(문장의 길이 혹은 단어 개수)에 상관없이 결과가 안정적인 편

from konlpy.tag import Komoran
import numpy as np
from numpy import dot
from numpy.linalg import norm

# 코사인 유사도 계산
def cos_sim(vec1, vec2) :
    return dot(vec1, vec2) / (norm(vec1) * norm(vec2))

# TDM(Term-Document Matrix, 단어 문서 행렬) 만들기
def make_term_doc_mat(sentence_bow, word_dics) :
    freq_mat = {}

    for word in word_dics:
        freq_mat[word] = 0

    for word in word_dics:
        if word in sentence_bow:
            freq_mat[word] += 1

    return freq_mat

# 단어 벡터 만들기
def make_vector(tdm):
    vec = []
    for key in tdm:
        vec.append(tdm[key])
    return vec

# 문장 정의
sentence1 = "6월에 뉴턴은 선생님의 제안으로 트리니티에 입학했다."
sentence2 = "6월에 뉴턴은 선생님의 제안으로 대학교에 입학했다."
sentence3 = "나는 맛있는 밥을 뉴턴 선생님과 함께 먹었다."

# 형태소 분석기 이용, 단어 묶음 리스트 생성
komoran = Komoran()
bow1 = komoran.nouns(sentence1)
bow2 = komoran.nouns(sentence2)
bow3 = komoran.nouns(sentence3)

# 단어 묶음 리스트 합치기
bow = bow1 + bow2 + bow3

# 단어 묶음에서 중복 제거, 단어 사전 구축
word_dics = []
for token in bow:
    if token not in word_dics:
        word_dics.append(token)

# 문장별 단어 문서 행렬 계산
freq_list1 = make_term_doc_mat(bow1, word_dics)
freq_list2 = make_term_doc_mat(bow2, word_dics)
freq_list3 = make_term_doc_mat(bow3, word_dics)
print(freq_list1)
print(freq_list2)
print(freq_list3)

# 문장 벡터 생성
doc1 = np.array(make_vector(freq_list1))
doc2 = np.array(make_vector(freq_list2))
doc3 = np.array(make_vector(freq_list3))

# 코사인 유사도 계산
r1 = cos_sim(doc1, doc2)
r2 = cos_sim(doc3, doc1)
print(r1)
print(r2)


{'6월': 1, '뉴턴': 1, '선생님': 1, '제안': 1, '트리니티': 1, '입학': 1, '대학교': 0, '밥': 0, '선생': 0, '님과 함께': 0}
{'6월': 1, '뉴턴': 1, '선생님': 1, '제안': 1, '트리니티': 0, '입학': 1, '대학교': 1, '밥': 0, '선생': 0, '님과 함께': 0}
{'6월': 0, '뉴턴': 1, '선생님': 0, '제안': 0, '트리니티': 0, '입학': 0, '대학교': 0, '밥': 1, '선생': 1, '님과 함께': 1}
0.8333333333333335
0.20412414523193154
